In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import os 
import time
root_logdir=os.path.join(os.curdir,'my_log')
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

In [ ]:
#Generate test time series data， batch_size different series and each series n_steps long;
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
    return series[..., np.newaxis].astype(np.float32)


#split into train,valid,test set
n_steps=50
series=generate_time_series(10000,n_steps+1)
X_train,y_train=series[:7000,:n_steps],series[:7000,-1]
X_valid,y_valid=series[7000:9000,:n_steps],series[7000:9000,-1]
X_test,y_test=series[9000:,:n_steps],series[9000:,-1]


#Performance baseline
y_pred=X_valid[:,-1]
baselineError=np.mean(keras.losses.mse(y_valid,y_pred))

In [ ]:
#Construct 1-layer RNN model
simpleRnn=keras.models.Sequential([
    keras.layers.Input(shape=[None,1]),
    keras.layers.SimpleRNN(1,return_sequences=True)
])

#Compile 
simpleRnn.compile(loss='mse',optimizer='Adam')

#Train
simpleRnn.fit(X_train,y_train,validation_data=[X_valid,y_valid],epochs=20)

In [ ]:
#plotting traning history
def plot_curve(history_data):
    pd.DataFrame(history_data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
    plt.show()
    return 

In [ ]:
#Deep RNN
deepRnn=keras.models.Sequential([
    keras.layers.Input(shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(1)
])

#Compile
deepRnn.compile(optimizer='Adam',loss='mse')


In [ ]:
deepRnn.fit(X_train,y_train,validation_data=[X_valid,y_valid],epochs=20)
pd.DataFrame(deepRnn.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, .1) # set the vertical range to [0-1]
plt.show()

In [ ]:
#Modified deep RNN
modRNN=keras.models.Sequential([
    keras.layers.Input(shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(1)
])
#compile
modRNN.compile(loss='mse',optimizer='adam')
#train
modRNN.fit(X_train,y_train,validation_data=[X_valid,y_valid],epochs=20)
pd.DataFrame(deepRnn.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, .02) # set the vertical range to [0-1]
plt.show()

In [ ]:
#split into train,valid,test set
n_steps=50
series=generate_time_series(10000,n_steps+10)
X_train,y_train=series[:7000,:n_steps],series[:7000,n_steps:]
X_valid,y_valid=series[7000:9000,:n_steps],series[7000:9000,n_steps:]
X_test,y_test=series[9000:,:n_steps],series[9000:,n_steps:]



In [ ]:
#RNN with multi-output
multiRnn=keras.models.Sequential([
    keras.layers.Input(shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(20),
    keras.layers.Dense(10)
])
#compile 
multiRnn.compile(loss='mse',optimizer='adam')
multiRnn.fit(X_train,y_train,validation_data=[X_valid,y_valid],epochs=20)

In [ ]:
#data preparation
Y=np.empty((10000,n_steps,10))
for steps in range(10):
    Y[:,:,steps]=series[:,steps+1:steps+1+n_steps,0]
Y_train=Y[:7000]
Y_valid=Y[7000:9000]
Y_test=Y[9000:]

In [ ]:
multiRnn=keras.models.Sequential([
    keras.layers.Input(shape=[None,1]),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.SimpleRNN(20,return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

def custom_mse(y_true,y_pred):
    return keras.metrics.mse(y_true[:-1],y_pred[:-1])


multiRnn.compile(loss='mse',optimizer=keras.optimizers.Adam(learning_rate=0.01),metrics=[custom_mse])

In [ ]:
run_log=get_run_logdir()
tensor_cb=keras.callbacks.TensorBoard(run_log)
multiRnn.fit(X_train,Y_train,validation_data=[X_valid,Y_valid],epochs=20,callbacks=[tensor_cb])

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006
!kill 19234

In [ ]:
pd.DataFrame(multiRnn.history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, .06) # set the vertical range to [0-1]
plt.show()

In [ ]:
#LSTM RNN
